# DerivaML Dataset Example.

DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

## Set up DerivaML  for test case

In [1]:
from dash.testing.errors import DashTestingError

from schema_setup.create_schema import define_table_dataset
%load_ext autoreload
%autoreload 2

In [13]:
import os
import pandas as pd
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml.schema_setup.test_catalog import create_test_catalog, DemoML
from deriva_ml.deriva_ml_base import MLVocab as vt
from deriva_ml.dataset_bag import DatasetBag
from IPython.display import display, Markdown

Set the details for the catalog we want and authenticate to the server if needed.

In [3]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


You are already logged in.


Create a test catalog and get an instance of the DemoML class.

In [4]:
test_catalog = create_test_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

## Configure DerivaML Datasets

In Deriva-ML a dataset is used to aggregate instances of entities.  However, before we can create any datasets, we must configure 
Deriva-ML for the specifics of the datasets.  The first stp is we need to tell Deriva-ML what types of use defined objects can be associated with a dataset.  

Note that out of the box, Deriva-ML is configured to allow datasets to contained dataset (i.e. nested datasets), so we don't need to do anything for that specific configuration.

In [5]:
print(f"Current dataset element types: {[a.name for a in ml_instance.list_dataset_element_types()]}")
ml_instance.add_dataset_element_type("Subject")
ml_instance.add_dataset_element_type("Image")
print(f"New dataset element types {[a.name for a in ml_instance.list_dataset_element_types()]}")

Current dataset element types: ['Dataset']
New dataset element types ['Dataset', 'Subject', 'Image']


Now that we have configured our datasets, we need to identify the dataset types so we can distiguish between them.

In [6]:
# Create a new dataset
ml_instance.add_term(vt.dataset_type, "DemoSet", description="A test dataset")
ml_instance.add_term(vt.dataset_type, 'Partitioned', description="A partitioned dataset for ML training.")
ml_instance.add_term(vt.dataset_type, "Subject", description="A test dataset")
ml_instance.add_term(vt.dataset_type, "Image", description="A test dataset")
ml_instance.add_term(vt.dataset_type, "Training", description="Training dataset")
ml_instance.add_term(vt.dataset_type, "Testing", description="Training dataset")
ml_instance.add_term(vt.dataset_type, "Validation", description="Validation dataset")

ml_instance.list_vocabulary_terms(vt.dataset_type)

[VocabularyTerm(name='DemoSet', synonyms=[], id='ml-test:36R', uri='/id/36R', description='A test dataset', rid='36R'),
 VocabularyTerm(name='Partitioned', synonyms=[], id='ml-test:36T', uri='/id/36T', description='A partitioned dataset for ML training.', rid='36T'),
 VocabularyTerm(name='Subject', synonyms=[], id='ml-test:36W', uri='/id/36W', description='A test dataset', rid='36W'),
 VocabularyTerm(name='Image', synonyms=[], id='ml-test:36Y', uri='/id/36Y', description='A test dataset', rid='36Y'),
 VocabularyTerm(name='Training', synonyms=[], id='ml-test:370', uri='/id/370', description='Training dataset', rid='370'),
 VocabularyTerm(name='Testing', synonyms=[], id='ml-test:372', uri='/id/372', description='Training dataset', rid='372'),
 VocabularyTerm(name='Validation', synonyms=[], id='ml-test:374', uri='/id/374', description='Validation dataset', rid='374')]

Now create datasets and populate with elements from the test catalogs.

In [7]:
system_columns = ['RCT', 'RMT', 'RCB', 'RMB']

subject_dataset = ml_instance.create_dataset(['DemoSet', 'Subject'], description="A subject dataset")
image_dataset = ml_instance.create_dataset(['DemoSet', 'Image'], description="A image training dataset")
datasets = pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)
display(datasets)

,RID,Description,Dataset_Type
0,376,A subject dataset,"[DemoSet, Subject]"
1,37C,A image training dataset,"[DemoSet, Image]"


And now that we have defined some datasets, we can add elements of the approproate type to them.  We can see what is in our new datasets by listing the dataset members.

In [8]:
dp = ml_instance.domain_path  # Each call returns a new path instance, so only call once...
subject_rids = [i['RID'] for i in dp.tables['Subject'].entities().fetch()]
image_rids = [i['RID'] for i in dp.tables['Image'].entities().fetch()]

ml_instance.add_dataset_members(dataset_rid=subject_dataset, members=subject_rids)
ml_instance.add_dataset_members(dataset_rid=image_dataset, members=image_rids)

# List the contents of our datasets, and lets not include columns like modify time.
display(pd.DataFrame(ml_instance.list_dataset_members(subject_dataset)['Subject']).drop(columns=system_columns))
display(pd.DataFrame(ml_instance.list_dataset_members(image_dataset)['Image']).drop(columns=system_columns))

,RID,Name
0,2ZG,Thing1
1,2ZJ,Thing2
2,2ZM,Thing3
3,2ZP,Thing4
4,2ZR,Thing5
5,2ZT,Thing6
6,2ZW,Thing7
7,2ZY,Thing8
8,300,Thing9
9,302,Thing10


,RID,URL,Filename,Description,Length,MD5,Name,Subject
0,30R,/hatrac/Image/a87499a167e6961d6c81f62f47e852b7...,test_2ZG.txt,A test image,31,a87499a167e6961d6c81f62f47e852b7,None,2ZG
1,30T,/hatrac/Image/b33e3f78223b87143f542530abd86d1e...,test_2ZJ.txt,A test image,31,b33e3f78223b87143f542530abd86d1e,None,2ZJ
2,30W,/hatrac/Image/0f17ea53ddc22ec6befe44abd89cb7d9...,test_2ZM.txt,A test image,30,0f17ea53ddc22ec6befe44abd89cb7d9,None,2ZM
3,30Y,/hatrac/Image/9292500baae06aec4f50142755790b15...,test_2ZP.txt,A test image,31,9292500baae06aec4f50142755790b15,None,2ZP
4,310,/hatrac/Image/a99fae2c67ab1b255dea81a68a7ecfb3...,test_2ZR.txt,A test image,31,a99fae2c67ab1b255dea81a68a7ecfb3,None,2ZR
5,312,/hatrac/Image/9090f2cbd9a50959c3cf6b8206bd7cf3...,test_2ZT.txt,A test image,32,9090f2cbd9a50959c3cf6b8206bd7cf3,None,2ZT
6,314,/hatrac/Image/3ad4f00c4edc023765dd1998ad16778a...,test_2ZW.txt,A test image,32,3ad4f00c4edc023765dd1998ad16778a,None,2ZW
7,316,/hatrac/Image/2d0e4bf553a29e3772738c2747278413...,test_2ZY.txt,A test image,32,2d0e4bf553a29e3772738c2747278413,None,2ZY
8,318,/hatrac/Image/3b0a8644673cdaed8d2efeb9e4a11622...,test_300.txt,A test image,32,3b0a8644673cdaed8d2efeb9e4a11622,None,300
9,31A,/hatrac/Image/1f41e1780620502a64c20c917dfc0c6e...,test_302.txt,A test image,31,1f41e1780620502a64c20c917dfc0c6e,None,302


## Create partitioned dataset

Now lets create some subsets of the origional dataset based on subject level metadata. We are going to create the subsets based on the metadata values of the subjects.  SO we will download the subject dataset and look at its metadata to figure out whow to partition the origional data. Since we are not going to look at the images, we use dowload_dataset_bag, rather than materialize_bag.

In [14]:
bag_path, bag_rid = ml_instance.download_dataset_bag(subject_dataset)
ml_instance.materialize_dataset_bag(subject_dataset)
dataset_bag = DatasetBag(bag_path)
print(f"Bag materialized to {bag_path}")

Bag materialized to /private/var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmp8bj35v2v/376_6b3147ffdea444aa8fc6c821be2652c0815bdd4b51b10c887a842c9a6e0cb272/Dataset_376


The domain model has two object: Subject and Images where an Image is associated with a subject, but a subject can have multiple images associated with it.  Lets look at the subjects and partition into test and training datasets.

In [16]:
# Get information about the subjects.....
subject_df = dataset_bag.get_table_as_dataframe('Subject')[['RID', 'Name']]
image_df = dataset_bag.get_table_as_dataframe('Image')[['RID', 'Subject', 'URL']]
metadata_df = subject_df.join(image_df, lsuffix="_subject", rsuffix="_image")
display(metadata_df)

SyntaxError: '[' was never closed (1984327912.py, line 3)

For ths example, lets partition the data based on the name of the subject.  Of course in real examples, we would do a more complex analysis in deciding
what subset goes into each data set.

In [ ]:
def thing_number(name: pd.Series) -> pd.Series:
    return name.map(lambda n: int(n.replace('Thing','')))

training_rids = metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 0]['RID_image'].tolist()
testing_rids =  metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 1]['RID_image'].tolist()
validation_rids = metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 2]['RID_image'].tolist()

print(f'Training images: {training_rids}')
print(f'Testing images: {testing_rids}')
print(f'Validation images: {validation_rids}')


Now that we know what we want in each dataset, lets create datasets for each of our partitioned elements along with a nested dataset to track the entire collection.

In [ ]:
nested_dataset = ml_instance.create_dataset(['Partitioned', 'Image'], description='A nested dataset for machine learning')
training_dataset = ml_instance.create_dataset('Training', description='An image dataset for training')
testing_dataset = ml_instance.create_dataset('Testing', description='A image dataset for testing')
validation_dataset = ml_instance.create_dataset('Validation', description='A image dataset for validation')
pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)

And then fill the datasets with the appropriate members.

In [ ]:

ml_instance.add_dataset_members(dataset_rid=nested_dataset, members=[training_dataset, testing_dataset, validation_dataset])
ml_instance.add_dataset_members(dataset_rid=training_dataset, members=training_rids)
ml_instance.add_dataset_members(dataset_rid=testing_dataset, members=testing_rids)
ml_instance.add_dataset_members(dataset_rid=validation_dataset, members=validation_rids)


Ok, lets see what we have now.

In [ ]:
display(Markdown('## Nested Dataset'))
display(pd.DataFrame(ml_instance.list_dataset_members(nested_dataset)['Dataset']).drop(columns=system_columns))
display(Markdown('## Training Dataset'))
display(pd.DataFrame(ml_instance.list_dataset_members(training_dataset)['Image']).drop(columns=system_columns))
display(Markdown('## Testing Dataset'))
display(pd.DataFrame(ml_instance.list_dataset_members(testing_dataset)['Image']).drop(columns=system_columns))
display(Markdown('## Validation Dataset'))
display(pd.DataFrame(ml_instance.list_dataset_members(validation_dataset)['Image']).drop(columns=system_columns))

As our very last step, lets get a PID that will allow us to share and and cite the dataset that we just created

In [ ]:
ml_instance.cite(nested_dataset)

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)